In [1]:
import numpy as np
from scipy.signal import filtfilt, butter, argrelextrema
from scipy import interpolate
from mysql import connector 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Достаем данные из БД
m_id = 3
conn = connector.connect(host='127.0.0.1', user='root', password='T0_Gather', database='NAVKA2')
cursor = conn.cursor()

## Детектирование Каскадов в данных

In [3]:
# Вспомогательный метод вместо MinMaxScaler
def minmax(a):
    result = []
    mn, mx = min(a), max(a)
    for i in a:
        result.append((i - mn) / (mx - mn))
    return np.array(result)

# Вспомогательный метод вместо фильтрация +  MinMaxScaler
fminmax = lambda d, f, s: minmax(abs(filtfilt(*butter(3, f, s), d, axis=0)))

In [4]:
def get_csc(acc, th1=2, f1=0.2, th2=0.3):
    # acc: np.array of sum acceleration
    # return: [[t0,tn]] - отметки времени начала и окончания каскада.  
    plt.plot(acc)
    dat = np.where(acc > th1, acc, 0)
    dat = minmax(dat)
    dat = fminmax(dat, f1, 'lp')

    dat = np.where(dat > th2, dat, 0)
    plt.plot(dat*25)
    # Определяем пары точек карахтеризующих номер времени начала и окончания каскада. 
    csc = []
    for i in range(1, len(dat)):
        if dat[i] == 0 and dat[i - 1] != 0:
            if len(csc) == 0:
                csc.append([0])
            csc[-1].append(i)
        elif dat[i] != 0 and dat[i - 1] == 0:
            csc.append([i])
    try:
        if len(csc[-1]) % 2 == 1:
            csc[-1].append(len(dat) - 1)
    except:
        return None
    return csc 

In [5]:
# Выгрузка данных для детектирования каскадов
sid = 3
plt.figure(figsize=(15,10))

cursor.execute(f"SELECT COUNT(tick) FROM vAcc WHERE sid={sid};")
# Номер последней записи в сессии
i_n = cursor.fetchall()[0][0] 

# Определение завершенности последнего каскада
cursor.execute(f"SELECT MAX(tn) FROM Cscds WHERE sid={sid}")
# Номер записи на которой закончился последний каскад
i_c = cursor.fetchall()[0][0] 

if i_n == i_c:     
    cursor.execute(f"SELECT MAX(t0) FROM Cscds WHERE sid={sid}")
    # Номер записи на которой начался последний каскад
    i_c = cursor.fetchall()[0][0]
    
i_c = i_c if i_c else 0
# Выбирается еще не обработанный набор данных 
cursor.execute(f"SELECT SQRT(x*x+y*y+z*z) FROM vAcc WHERE sid={sid} ORDER BY tick DESC LIMIT {i_n-i_c} ;")
dat = np.array([data for data in cursor][::-1][21600:22100]).T

# Анализ "хвоста данных"
if dat.any():
    csc = get_csc(dat[-1],3, 0.001, 0.1)
    
    
    csc = [[i+i_c for i in j] for j in csc] if csc else None
    
    # Дополнительная защита отт разрывов
    #if csc[-1][1]+5>i_n:
    #    csc.remove(csc[-1])
    print(csc)

<Figure size 1080x720 with 0 Axes>

In [140]:
# Перезапись каскадов
if csc:
    # Если каскад не заканчивается в самом конце тренда
    #if csc[-1][1] == end_index:
    #    csc = csc[:-1]
    if len(csc) > 0:
        query = "INSERT INTO Cscds (sid, t0, tn) Values "
        for c in csc[:-1]:
           query+= f"{sid,c[0],c[-1]}, "
        query += f"{sid,csc[-1][0],csc[-1][-1]}; "
        print(query)
        cursor.execute(query)
        print('executed')
        conn.commit()

INSERT INTO Cscds (sid, t0, tn) Values (3, 20863, 22608), (3, 33711, 35437), (3, 103522, 104375), (3, 111393, 113177), (3, 127949, 129718), (3, 143321, 145103), (3, 150508, 151848), (3, 163469, 165261), (3, 178866, 180034), (3, 187616, 189369), (3, 201815, 203579); 
executed


## Детектирование Прыжков в каскадах

In [150]:
def get_jmps(data):
    result = data.copy(deep=True)
    
    t = [int(i) for i in data.index]
    ti = list(range(t[0],t[-1],1))

    data = pd.merge(pd.DataFrame(index=ti), data,left_index=True, right_index=True, how='outer')
    data = data.interpolate()
    
    plt.plot(data)
    #strt = int(data[data['A']==max(data['A'])].index[0])
    #end = int(data[data['G']==max(data['G'])].index[0])
    
    #data = data.loc[end:]
    #ex = argrelextrema(data['G'].values, np.greater, order = 200)[0]
    #data = data.loc[[end+i for i in ex]]
    #end = int(data[data['G']==max(data['G'])].index[0])
    
    #if end-strt<2000:
    #    return []
    
    # На будущее когда будет несколько прыжков пока делаю массив Series
    #return [result.loc[strt:end]]
    return 0

In [177]:
# Выгрузка данных для детектироания прыжков
sid = 3
cursor.execute(f"SELECT cid, t0, tn FROM Cscds WHERE sid={sid}")
csc = cursor.fetchall()
ca = csc[0]
print(csc)
# Достаем данные за все время (пока так)
cid,i_0,i_n = ca
cursor.execute(f"SELECT tick FROM vAcc WHERE sid={sid}")
ticks = [d[0] for d in cursor][i_0:i_n]

# Достаем для каскада данные по акселерометру
cursor.execute(f"SELECT tick, SQRT(x*x+y*y+z*z) FROM vAcc WHERE sid={sid} AND tick>={min(ticks)} AND tick<={max(ticks)}")
A = pd.DataFrame(np.array([d for d in cursor]),columns= ['t','a'])
A
#plt.plot(A['t'],A['a'])
#get_jmps(A)

[(767, 20863, 22608), (768, 33711, 35437), (769, 103522, 104375), (770, 111393, 113177), (771, 127949, 129718), (772, 143321, 145103), (773, 150508, 151848), (774, 163469, 165261), (775, 178866, 180034), (776, 187616, 189369), (777, 201815, 203579)]


,t,a
0,3062.0,1.239900
1,3062.0,1.263842
2,3062.0,1.280355
3,3062.0,1.291686
4,3062.0,1.319831
5,3062.0,1.343076
6,3062.0,1.347116
7,3063.0,1.340944
8,3063.0,1.327162
9,3063.0,1.308548


In [144]:
def get_jmps(data):
    result = data.copy(deep=True)
    
    t = [int(i) for i in data.index]
    ti = list(range(t[0],t[-1],1))

    data = pd.merge(pd.DataFrame(index=ti), data,left_index=True, right_index=True, how='outer')
    data = data.interpolate()
    
    strt = int(data[data['A']==max(data['A'])].index[0])
    end = int(data[data['G']==max(data['G'])].index[0])
    
    data = data.loc[end:]
    ex = argrelextrema(data['G'].values, np.greater, order = 200)[0]
    data = data.loc[[end+i for i in ex]]
    end = int(data[data['G']==max(data['G'])].index[0])
    
    if end-strt<2000:
        return []
    
    # На будущее когда будет несколько прыжков пока делаю массив Series
    return [result.loc[strt:end]]

In [143]:
# Выгрузка данных для детектироания прыжков
sid = 3
cursor.execute(f"SELECT cid, t0, tn FROM Cscds WHERE sid={sid}")
csc = cursor.fetchall()
ca = csc[0]
print(csc)
# Достаем данные за все время (пока так)
cid,i_0,i_n = ca
cursor.execute(f"SELECT tick FROM vAcc WHERE sid={sid}")
ticks = [d[0] for d in cursor][i_0:i_n]

# Достаем для каскада данные по акселерометру
cursor.execute(f"SELECT SQRT(x*x+y*y+z*z) FROM vAcc WHERE sid={sid} AND tick>={min(ticks)} AND tick<={max(ticks)}")
A = pd.DataFrame(np.array([d for d in cursor]),columns= ['a'])
A['t'] = ticks

# Достаем для каскада данные по гироскопу (тики не синхронизированны)
cursor.execute(f"SELECT tick, SQRT(x*x+y*y+z*z) FROM vGyro WHERE sid={sid} AND tick>={min(ticks)} AND tick<={max(ticks)}")
G = pd.DataFrame(np.array([d for d in cursor]), columns= ['t','g'])

# Объединяем в единый DataFrame
data = pd.merge(A, G, on='t', how='outer')
data.columns = ['A','t','G']
data.index = data['t']
data = data.drop('t', axis=1)
data = data.sort_index()
data = data.interpolate().dropna()

result = get_jmps(data)
for jump in result:
    #cursor.execute(f"INSERT INTO Jmps (cid) VALUE ({cid})")
    #cursor.execute(f"SELECT jid FROM Jmps ORDER BY jid DESC LIMIT 1")
    #jid = cursor.fetchall()[0][0]
    jid=7
    print(jid)
    to_insert = []
    for i in jump.index:
        val = jump.loc[i].values
        to_insert.append((jid,i,val[0],val[1]))   
    #cursor.execute(f"INSERT INTO vJmps (jid, tick, a,g) VALUES {str(to_insert)[1:-1]}")

[(767, 20863, 22608), (768, 33711, 35437), (769, 103522, 104375), (770, 111393, 113177), (771, 127949, 129718), (772, 143321, 145103), (773, 150508, 151848), (774, 163469, 165261), (775, 178866, 180034), (776, 187616, 189369), (777, 201815, 203579)]


ValueError: Length of values does not match length of index

In [189]:
conn.commit()
conn.close()